In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
# Read recipe inputs
flight_joined_prepared = dataiku.Dataset("flight_data")
df = flight_joined_prepared.get_dataframe()

In [0]:
#-----------------------------------------------------------------
# Dataset Settings
#-----------------------------------------------------------------

# Select a subset of features to use for training
SCHEMA = {
    'target': 'Late',
    'features_num': ['dep_month', 'dep_woy', 'dep_hour','Distance','Late_avg'],
    'features_cat': ['UniqueCarrier', 'Origin','Dest']
}

In [0]:
#-----------------------------------------------------------------
# Preprocessing on Training Set
#-----------------------------------------------------------------

# Numerical variables
df_num = df[SCHEMA['features_num']]

trf_num = Pipeline([
    ('imp', SimpleImputer(strategy='mean')),
    ('sts', StandardScaler()),
])

# Categorical variables
df_cat = df[SCHEMA['features_cat']]

trf_cat = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", trf_num, SCHEMA['features_num']),
        ("cat", trf_cat, SCHEMA['features_cat'])
    ]
)

In [0]:
import dataiku
import mlflow


project = dataiku.api_client().get_default_project()
managed_folder = project.get_managed_folder('3hOB5aod')

with project.setup_mlflow(managed_folder=managed_folder) as mlflow:
    mlflow.set_experiment("experiment_2")

    # activate Mflow autologging
    mlflow.sklearn.autolog()

    with mlflow.start_run(run_name="my_run"):
        clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

        param_grid = {
            "clf__max_depth"        : [3],
            "clf__max_features"     : [1],
            "clf__min_samples_split": [2],
            "clf__min_samples_leaf" : [1],
            "clf__bootstrap"        : [False],
            "clf__criterion"        : ["gini"],
            "clf__n_estimators"     : [10]
        }

        gs = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1, scoring='roc_auc', cv=3)
        X = df[SCHEMA['features_num'] + SCHEMA['features_cat']]
        Y = df[SCHEMA['target']].values
        gs.fit(X, Y)
        clf = gs.best_estimator_

In [0]:
client = dataiku.api_client()
project = client.get_project('MLFLOWO16N')
# Get or create saved models
if dataiku.get_custom_variables()["saved_model_id"] == "":
    saved_model = project.create_mlflow_pyfunc_model("mlflow_model", "BINARY_CLASSIFICATION")
    project.update_variables({"saved_model_id": saved_model.id})
else:
    saved_model = project.get_saved_model(dataiku.get_custom_variables()["saved_model_id"])

In [0]:
mlflow_version = saved_model.import_mlflow_version_from_managed_folder(dataiku.get_custom_variables()["custom_model_version"], "3hOB5aod", 'experiment_2/my_run_jHL/artifacts/model/',code_env_name="mlflow_latest")
project.update_variables({"custom_model_version": int(dataiku.get_custom_variables()["custom_model_version"]) + 1})

In [0]:
mlflow_version.set_core_metadata(SCHEMA['target'], class_labels=["False", "True"], get_features_from_dataset="flight_ground_truth_prepared")
mlflow_version.evaluate("flight_ground_truth_prepared")

In [5]:
import dataiku
handle = dataiku.Folder("3hOB5aod")
# pass a partition identifier if the folder is partitioned
paths = handle.list_paths_in_partition()
paths

['/experiment_2/my_run/artifacts/best_estimator/MLmodel',
 '/experiment_2/my_run/artifacts/best_estimator/conda.yaml',
 '/experiment_2/my_run/artifacts/best_estimator/model.pkl',
 '/experiment_2/my_run/artifacts/best_estimator/requirements.txt',
 '/experiment_2/my_run/artifacts/cv_results.csv',
 '/experiment_2/my_run/artifacts/model/MLmodel',
 '/experiment_2/my_run/artifacts/model/conda.yaml',
 '/experiment_2/my_run/artifacts/model/model.pkl',
 '/experiment_2/my_run/artifacts/model/requirements.txt',
 '/experiment_2/my_run_4Nm/artifacts/best_estimator/MLmodel',
 '/experiment_2/my_run_4Nm/artifacts/best_estimator/conda.yaml',
 '/experiment_2/my_run_4Nm/artifacts/best_estimator/model.pkl',
 '/experiment_2/my_run_4Nm/artifacts/best_estimator/python_env.yaml',
 '/experiment_2/my_run_4Nm/artifacts/best_estimator/requirements.txt',
 '/experiment_2/my_run_4Nm/artifacts/cv_results.csv',
 '/experiment_2/my_run_4Nm/artifacts/estimator.html',
 '/experiment_2/my_run_4Nm/artifacts/model/MLmodel',
 

In [6]:
with handle.get_download_stream("experiment_2/my_run_jHL/artifacts/model/model.pkl") as f:
    data = f.read()

In [10]:
output_handle = dataiku.Folder("WISPen39")

with output_handle.get_writer(data) as w:
    w.write("test_output.pkl")

ERROR:root:Pipe to generator thread failed
Traceback (most recent call last):
  File "/home/dataiku/dataiku-dss-11.0.3/python/dataiku/core/dkuio.py", line 253, in run
    self.consumer(self._generate())
  File "/home/dataiku/dataiku-dss-11.0.3/python/dataiku/core/managed_folder.py", line 38, in upload_call
    jek_or_backend_void_call("managed-folders/upload-path", params={
  File "/home/dataiku/dataiku-dss-11.0.3/python/dataiku/core/intercom.py", line 446, in jek_or_backend_void_call
    return backend_void_call(path, data, err_msg, **kwargs)
  File "/home/dataiku/dataiku-dss-11.0.3/python/dataiku/core/intercom.py", line 437, in backend_void_call
    return _handle_void_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
  File "/home/dataiku/dataiku-dss-11.0.3/python/dataiku/core/intercom.py", line 495, in _handle_void_resp
    raise Exception("%s: %s" % (err_msg, _get_error_message(err_data).encode("utf8")))
Exception: None: b'<h1>Bad Message 414</h1><pre>reason: UR

Exception: Error : None: b'<h1>Bad Message 414</h1><pre>reason: URI Too Long</pre>'